In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
data_dir = '/Users/xiaoqianxiao/UKB/data'
participantsInfo_file = 'participants.csv'
participantsInfo_file_path = os.path.join(data_dir,participantsInfo_file)
participantsInfo = pd.read_csv(participantsInfo_file_path)

In [36]:
participantsInfo.head()

,eid,p20436,p20439,p20440,p20446,p20441,p20449,p20536,p20532,p20435,...,p31016_i2,p31016_i3,p31018_i2,p31018_i3,p31019_i2,p31019_i3,p31015_i2,p31015_i3,p31014_i2,p31014_i3
0,1000011,3.0,2.0,2.0,1.0,1.0,1.0,2.0,-121.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000047,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,1000047_31016_2_0.zip,NaN,1000047_31018_2_0.zip,NaN,1000047_31019_2_0.zip,NaN,1000047_31015_2_0.zip,NaN,1000047_31014_2_0.zip,NaN
2,1000190,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000240,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1000258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
participantsInfo.columns

Index(['eid', 'p20436', 'p20439', 'p20440', 'p20446', 'p20441', 'p20449',
       'p20536', 'p20532', 'p20435', 'p20450', 'p20437', 'p20421', 'p20420',
       'p20538', 'p20425', 'p20542', 'p20543', 'p20540', 'p20541', 'p20539',
       'p20537', 'p20418', 'p20426', 'p20423', 'p20429', 'p20419', 'p20422',
       'p20417', 'p20427', 'p20514', 'p20510', 'p20517', 'p20519', 'p20511',
       'p20507', 'p20508', 'p20518', 'p20513', 'p20506', 'p20509', 'p20520',
       'p20515', 'p20516', 'p20505', 'p20512', 'p41270', 'p41271', 'p31016_i2',
       'p31016_i3', 'p31018_i2', 'p31018_i3', 'p31019_i2', 'p31019_i3',
       'p31015_i2', 'p31015_i3', 'p31014_i2', 'p31014_i3'],
      dtype='object')

In [32]:
# transfer EA
dl = []
for i in range(participantsInfo.shape[0]):
    # Prefer not to answer
    if participantsInfo['6138-2.0'].iloc[i] == -3:
        dl.append(participantsInfo.index[i])
    if participantsInfo['6138-2.0'].iloc[i] == 1:
        participantsInfo['6138-2.0'].iloc[i] = 20
    if participantsInfo['6138-2.0'].iloc[i] == 2:
        participantsInfo['6138-2.0'].iloc[i] = 13
    if participantsInfo['6138-2.0'].iloc[i] == 3:
        participantsInfo['6138-2.0'].iloc[i] = 10
    if participantsInfo['6138-2.0'].iloc[i] == 4:
        participantsInfo['6138-2.0'].iloc[i] = 10
    if participantsInfo['6138-2.0'].iloc[i] == 5:
        participantsInfo['6138-2.0'].iloc[i] = 19 
    if participantsInfo['6138-2.0'].iloc[i] == 6:
        participantsInfo['6138-2.0'].iloc[i] = 15
    # None of the above
    if participantsInfo['6138-2.0'].iloc[i] == -7:
        dl.append(participantsInfo.index[i])   
participantsInfo.drop(dl,inplace=True)

KeyError: '6138-2.0'

In [38]:
# select individuals with depression or anxiety 
l1 = []
l2 = []
# for i in range(participantsInfo.shape[0]):
#     if participantsInfo['2050-2.0'].iloc[i] >= 3 or participantsInfo['2060-2.0'].iloc[i] >= 3:
#         l1.append(participantsInfo['eid'].iloc[i])
#     if participantsInfo['2070-2.0'].iloc[i] >= 3:
#         l2.append(participantsInfo['eid'].iloc[i])
for i in range(participantsInfo.shape[0]):
    if participantsInfo['p2050'].iloc[i] >= 3 or participantsInfo['p2060'].iloc[i] >= 3:
        l1.append(participantsInfo['eid'].iloc[i])
    if participantsInfo['p2070'].iloc[i] >= 3:
        l2.append(participantsInfo['eid'].iloc[i])

KeyError: 'p2050'

In [39]:
help(name_regex)

NameError: name 'name_regex' is not defined